# introduction

The Black-Litterman (BL) asset allocation model is pretty amazing. It consists of three brilliant ideas, whose interplay is a brilliant idea on its own. It is (1) a Bayesian econometrics model (2) with a very smart choice of the prior and the data, and (3) an asset allocation model. Neither of the ideas feels out of place, and neither was introduced just for the sake of it.

In this notebook, I would like to visualize the Bayesian dimension of the BL model, as pointed out (among others) by [Kolm and Ritter (2017)](https://cims.nyu.edu/~ritter/kolm2017bayesian.pdf). I take the example from He and Litterman (2009), use `Turing.jl` to set up and numerically solve a Bayesian update model, arriving at the same values as in the closed-form solution derived by Black and Litterman. 

# model recap
Black and Litterman start by pondering how a portfolio manager could incorporate (possibly contradicting) views about expected returns, as coming e.g from different analysts, into the mean-variance optimization problem. Neat! They approach the problem by making a number of assumptions, and derive a closed-form solution.

# black-litterman model through bayesian lens
Now, how does the Bayesian connection come about? The answer is: by treating expected returns as a random variable, making a prior belief about its distribution, and updating this belief in the face of new data that is the investors' views! The rest is just plugging in the updated estimates into the asset allocation problem.

Here are the steps in detail:

1. There are several assets, whose market capitalization dictates the 'market', or 'equilibrium' portfolio $w_m$ &ndash; why this portfolio is important will become clear later;
2. the covariance matrix $\Sigma_r$ of asset returns is assumed to be known or at least estimated with very high precision;
3. the expected returns $\mu_r$, however, are a vector-valued random variable, _initially believed_ to follow the (multivariate) Normal distribution with mean $\Pi$ and covariance $\Sigma_{\mu} = \tau \Sigma_r$; $\Pi$ is taken to be the expected returns which would without any other information result in the market portfolio being mean-variance efficient; in other words, given $\Pi$, $w_m$ is the solution to: 
    $$
        \underset{w}{\max} \ w'\Pi - \frac{\delta}{2} w'\Sigma_r w
    $$
    for some value of $\delta$.
4. the views are kind of data points which can tell a little more about the distribution of $\mu$; the views can be direct or indirect, but always possible to express as 
    $$
        P \mu = q + \varepsilon_q,
    $$ 
    where $\varepsilon$ is a random variable introducing the noise (of course, Normally distributed!)
5. finally, the update of the prior belief from (3) is made based on the views in the Bayesian fashion: intuitively, the updated distribution will reflect the views, the stronger so, the more numerous and less noisy they are;
6. finally, we don't need the whole updated distribution of $\mu_r$, but only its mean because, nesting Normal random variables preserves the mean of the innermost variable, and because, bar the covariance, _nothing else matters_ for the mean-variance optimization.

Black and Litterman provide a closed-form solution for the mean of the asset returns (which turns out to be the mean of the updated distribution of $\mu_r$) and their covariance (which turns out to be updated, too). The solution is derived from some simple mathematics without references to prior, conjugates, posterior, likelihoods and all that Bayesian stuff...

# implementation as a bayesian update problem
...but where is fun is that? Let's solve the Bayesian problem numerically using the quite amazing `Turing.jl` library, and confirm what I claim above!

First, recreate the environment, where `Turing` and other necessary packages are installed, and import them.

In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/projects/black-litterman-bayes`


In [2]:
using Turing, Distributions, MCMCChains

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Then, let's import some helper function, like the ones fetching the data from He and Litterman (1999).

In [3]:
# datafeed functions and nearest_pd_matrix
include("src_julia/datafeed.jl")
include("src_julia/utilities.jl")

nearest_pd_matrix (generic function with 1 method)

Now, let's load data from that paper: the covariance matrix is constructed using the correlation structure from Table 1 and the standard deviations from Table 2; the market portfolio weights are from Table 2.

In [20]:
# covariance and market weights
Σ_r = get_covariance();

# vcv can be numerically non-hermitian, so let's fix it
Σ_r = nearest_pd_matrix(Σ_r);

# Pi, the mean of expected returns' distribution
Π = get_table(2)[:, :Pi] / 100;

n_assets = length(Π);

println("covariance:"); flush(stdout)
display(round.(Σ_r, digits=4))

covariance:


7×7 Matrix{Float64}:
 0.0256  0.0159  0.019   0.0223  0.0148  0.0164  0.0147
 0.0159  0.0412  0.0334  0.036   0.0132  0.0247  0.0296
 0.019   0.0334  0.0615  0.0579  0.0185  0.0388  0.031
 0.0223  0.036   0.0579  0.0734  0.0201  0.0421  0.0331
 0.0148  0.0132  0.0185  0.0201  0.0441  0.017   0.012
 0.0164  0.0247  0.0388  0.0421  0.017   0.04    0.0244
 0.0147  0.0296  0.031   0.0331  0.012   0.0244  0.035

Also, define parameters as in the paper: 
  - the market risk aversion $\delta$ set to 0.025;
  - the scaling factor $\tau$ for the covariance in the prior distribution of the $\mu_r$ is set to 0.05; 
  - the variance of the view about the German market $\omega$ is set via $\omega/\tau = 0.021$.

In [21]:
# market risk aversion
δ = 2.5;

# variance scaling for the expected returns prior
τ = 0.05;

# (relative) view uncertainty
omega_through_tau = 0.021;

# view uncertainty
ω = omega_through_tau * τ;

Finally, define the necessary $P$ and $Q$ matrices for the view about the German market outperforming the rest of Europe by 5%; $P$ is taken from Table 4. Since we are dealing with the single view, it's easier to define $P$ and $Q$ as vectors:

In [22]:
# view that ger outperformce market cap-weighted fra+gbr by 5% (from table 3)
p = get_table(4)[:, :p] / 100;
q = 0.05;

println("long-short portfolio with the view on german market:")
println(p)

long-short portfolio with the view on german market:
[0.0, 0.0, -0.295, 1.0, 0.0, -0.705, 0.0]


Now, let's set up a Bayesian model by:

1. treating the expected returns of the assets as a random variable with a prior distribution;
2. treating the view about the German market as the realization of a random variable connected to the expected returns;
3. adjusting the distribution of returns.

$$
\begin{gather}
    \mu_r \sim N(\Pi, \tau \Sigma_r) \\
    q | \mu_r \sim N(P'\mu_r, \omega) \\
    r | \mu \sim N(\mu_r, \Sigma),
\end{gather}
$$

where (1) defines the prior distribution of the expected returns means that the expected returns are initially

In [23]:
# black-litterman bayesian model
@model function bl(_q)

    global Π, Σ_r, p, ω

    # prior on mu
    μ_r ~ MvNormal(Π, τ* Σ_r);

    # given mu, the views are distributed as:
    _q ~ Normal(dot(p, μ_r), sqrt(ω));
    
    # condition the model on data
    return _q
end

bl (generic function with 2 methods)

...and sample from the posterior:

In [24]:
chain = sample(bl(q), NUTS(), 40000, progress = false);

┌ Info: Found initial step size
│   ϵ = 0.0125
└ @ Turing.Inference /home/ipozdeev/.julia/packages/Turing/JVSRF/src/mcmc/hmc.jl:191


In [27]:
post_est, _ = describe(chain);

μ_bar = round.(post_est[:, :mean] * 100, digits=1);

μ_bar_exog = get_table(4)[:, :mu_bar];
print(DataFrame(:μ_bar => μ_bar, :μ_bar_exog => μ_bar_exog))

7×2 DataFrame
 Row │ μ_bar    μ_bar_exog 
     │ Float64  Float64    
─────┼─────────────────────
   1 │     4.3         4.3
   2 │     7.6         7.6
   3 │     9.3         9.3
   4 │    11.0        11.0
   5 │     4.5         4.5
   6 │     6.9         7.0
   7 │     8.1         8.1